In [ ]:
import boto3

def transcribe_audio(input_audio_path):
    transcribe_client = boto3.client('transcribe')

    # Start transcription job
    response = transcribe_client.start_transcription_job(
        TranscriptionJobName='your-transcription-job-name',
        LanguageCode='en-US',  # Change to the appropriate language code
        MediaFormat='mp3',     # Change to the appropriate audio format
        Media={
            'MediaFileUri': f's3://your-bucket-name/{input_audio_path}'
        }
    )

    transcription_job_name = response['TranscriptionJob']['TranscriptionJobName']

    # Wait for the transcription job to complete
    transcribe_client.get_waiter('transcription_job_completed').wait(
        TranscriptionJobName=transcription_job_name
    )

    # Get the transcript
    response = transcribe_client.get_transcription_job(
        TranscriptionJobName=transcription_job_name
    )
    
    transcript_uri = response['TranscriptionJob']['Transcript']['TranscriptFileUri']
    
    return transcript_uri

def download_transcript(transcript_uri, output_text_path):
    transcribe_client = boto3.client('s3')

    # Download the transcript file
    transcribe_client.download_file(
        Bucket='your-bucket-name',
        Key=transcript_uri.split('/')[-1],  # Extract the file name from the URL
        Filename=output_text_path
    )

if __name__ == "__main__":
    input_audio_path = 'path/to/your/input/audio.mp3'
    output_text_path = 'path/to/save/output/transcript.txt'
    
    transcript_uri = transcribe_audio(input_audio_path)
    download_transcript(transcript_uri, output_text_path)
    
    print("Transcription complete. Text saved to:", output_text_path)